In [28]:
# import sys
# sys.path.append("d:\\BaiduNetdiskWorkspace\\2-code\\FundResearch\\")
import pandas as pd
import numpy as np
import matplotlib as plt
import warnings
from time import sleep

import quant_utils.data_moudle as dm
from quant_utils.constant import DB_CONN_LOCAL_MYSQL, DB_CONN_DATAYES
from quant_utils.utils import offset_report_date

plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False
warnings.filterwarnings("ignore")

In [29]:
def get_stats(trade_dt):
    perf_sql = f"""
    SELECT
        b.SECURITY_ID,
        b.SEC_SHORT_NAME,
        b.TICKER_SYMBOL,
        END_DATE,
        f.VALUE_NAME_CN,
        ANNUAL_TOTAL_RETURN,
        MAX_DRAWDOWN,
        ANNUAL_TOTAL_RETURN / ANNUAL_TOTAL_RISK AS SR,
        CALMAR_RATIO
    FROM
        fund_perf_indic2 a
        JOIN md_security b ON a.SECURITY_ID = b.SECURITY_ID
        LEFT JOIN sys_code f ON a.WINDOW = f.VALUE_NO
        AND f.CODE_TYPE_ID = 40106
    WHERE
        END_DATE = '{trade_dt}'
        AND WINDOW in (2, 3, 5, 6)
    ORDER BY WINDOW, SR
    """
    perf_df = DB_CONN_DATAYES.exec_query(perf_sql)

    report_date = offset_report_date(trade_dt, 4)[-1]
    fund_type_sql = f"""
    SELECT
        TICKER_SYMBOL,
        LEVEL_2
    FROM
        fund_type_own
    WHERE
        REPORT_DATE = '{report_date}'
        and LEVEL_1 in ('固收', '固收+')
        AND (
            (
                LEVEL_2 = '短债'
            AND TICKER_SYMBOL IN ( SELECT TICKER_SYMBOL FROM fund_redeem_fee WHERE 7d = 0 ))
            OR (
                LEVEL_2 != '短债'
            AND TICKER_SYMBOL IN ( SELECT TICKER_SYMBOL FROM fund_redeem_fee WHERE 30d = 0 ))
        )
    """
    fund_type = DB_CONN_LOCAL_MYSQL.exec_query(fund_type_sql)

    perf_df_bond = perf_df.merge(fund_type)
    # perf_df_bond.dropna(inplace=True)
    perf_df_bond.index = range(0, len(perf_df_bond.index))

    result = perf_df_bond[['SEC_SHORT_NAME', 'TICKER_SYMBOL', 'END_DATE', 'VALUE_NAME_CN', 'LEVEL_2']]
    cols = ['ANNUAL_TOTAL_RETURN','MAX_DRAWDOWN', 'SR', 'CALMAR_RATIO']

    result[cols] = perf_df_bond.groupby(by=['VALUE_NAME_CN', 'LEVEL_2'])[cols].rank(pct=True)
    result['MAX_DRAWDOWN'] = 1 - result['MAX_DRAWDOWN']
    result['total_score'] = result[cols].sum(axis=1)

    result_list = []
    for type in result['LEVEL_2'].unique():
        temp = result[result['LEVEL_2'] == type]
        result_score = temp[['TICKER_SYMBOL', 'SEC_SHORT_NAME', 'LEVEL_2']].drop_duplicates(subset=['TICKER_SYMBOL'])
        for col in ['ANNUAL_TOTAL_RETURN','MAX_DRAWDOWN', 'SR', 'CALMAR_RATIO', 'total_score']:

            temp_df = temp.pivot_table(index='TICKER_SYMBOL', columns='VALUE_NAME_CN', values=col)
            if len(temp_df.columns) < 4:
                continue
            temp_df['总分'] = temp_df['近半年']*0.2 + temp_df['近三个月']*0.2 + temp_df['近一年'] * 0.3 + temp_df['近两年'] * 0.3
            temp_df.columns = [ i + "_" + col  for i in temp_df.columns]
            result_score = result_score.merge(temp_df, on=['TICKER_SYMBOL'])
            result_list.append(result_score)
    score_df = pd.concat(result_list)
    score_df.dropna(inplace=True)
    score_df['排名'] = score_df.groupby(by=['LEVEL_2'])['总分_total_score'].rank(pct=True, ascending=False)*100

    return score_df

In [30]:
# fund_type_sql = """
# SELECT
# 	TICKER_SYMBOL,
# 	LEVEL_2
# FROM
# 	fund_type_own
# WHERE
# 	REPORT_DATE = '20220630'
#     and LEVEL_1 in ('固收', '固收+')
# """
# and TICKER_SYMBOL in (SELECT TICKER_SYMBOL FROM fund_redeem_fee WHERE 30d = 0)


In [31]:
trade_dts = []
for year in range(2016, 2022):
    for month in ['0331', '0630', '0930', '1231']:
        trade_dts.append(dm.offset_period_trade_dt(str(year) + month, -1, period='d'))
trade_dts.append(dm.offset_period_trade_dt('20220331', -1, period='d'))
trade_dts.append(dm.offset_period_trade_dt('20220630', -1, period='d'))
trade_dts.append(dm.offset_period_trade_dt('20220929', -1, period='d'))

In [32]:
stats_result_list = []
for i in range(0, len(trade_dts) - 1):
    start_date = trade_dts[i]
    end_date = trade_dts[i + 1]
    print(start_date + "开始计算")
    print("++"*20)
    df = get_stats(trade_dts[i])
    result_list = []
    for i, j in df.groupby('LEVEL_2'):
        temp = j.copy()
        temp['排名分组'] = pd.qcut(j['排名'], 5, [1,2,3,4,5])
        result_list.append(temp[['TICKER_SYMBOL', 'SEC_SHORT_NAME', 'LEVEL_2', '排名', '排名分组']])
    result = pd.concat(result_list)

    result['收益'] = result.apply(
        lambda s: np.log(dm.get_fund_adj_nav(s['TICKER_SYMBOL'], start_date=end_date, end_date=end_date)['ADJUST_NAV']/
            dm.get_fund_adj_nav(s['TICKER_SYMBOL'], start_date=start_date, end_date=start_date)['ADJUST_NAV']),
        axis=1
    )
    stats_result = (result.groupby(by=['LEVEL_2', '排名分组'])['收益'].median()).reset_index()
    stats_result['start_date'] = start_date
    stats_result['end_date'] = end_date
    stats_result_list.append(stats_result)
    sleep(5)
stats_result_df = pd.concat(stats_result_list)


20160330开始计算
++++++++++++++++++++++++++++++++++++++++
20160629开始计算
++++++++++++++++++++++++++++++++++++++++
20160929开始计算
++++++++++++++++++++++++++++++++++++++++
20161229开始计算
++++++++++++++++++++++++++++++++++++++++
20170330开始计算
++++++++++++++++++++++++++++++++++++++++
20170629开始计算
++++++++++++++++++++++++++++++++++++++++
20170928开始计算
++++++++++++++++++++++++++++++++++++++++
20171228开始计算
++++++++++++++++++++++++++++++++++++++++
20180329开始计算
++++++++++++++++++++++++++++++++++++++++
20180628开始计算
++++++++++++++++++++++++++++++++++++++++
20180927开始计算
++++++++++++++++++++++++++++++++++++++++
20181227开始计算
++++++++++++++++++++++++++++++++++++++++
20190328开始计算
++++++++++++++++++++++++++++++++++++++++
20190627开始计算
++++++++++++++++++++++++++++++++++++++++
20190927开始计算
++++++++++++++++++++++++++++++++++++++++
20191230开始计算
++++++++++++++++++++++++++++++++++++++++
20200330开始计算
++++++++++++++++++++++++++++++++++++++++
20200629开始计算
++++++++++++++++++++++++++++++++++++++++
20200929开始计算
+++++++++++++++

In [33]:

lsit = []
lsit_ret = []
for i, j in stats_result_df.groupby('LEVEL_2'):
    temp = j.pivot_table(index='end_date', columns='排名分组', values='收益')
    temp.loc[trade_dts[0], :] = 0
    temp = temp.sort_index()
    temp['1-5'] = temp[1] - temp[5]
    temp_nav = (np.exp(temp)).cumprod()
    temp_nav = temp_nav.reset_index()
    temp_nav['类型'] = i
    temp['类型'] = i
    lsit.append(temp_nav)
    lsit_ret.append(temp)
result = pd.concat(lsit)
ret_df = pd.concat(lsit_ret)

result.to_excel('./分组结果1.xlsx')
ret_df.to_excel('./分组收益1.xlsx')

In [34]:
ret_df.groupby(by=['类型'])['1-5'].describe()

,count,mean,std,min,25%,50%,75%,max
类型,,,,,,,,
中长期纯债型基金,27.0,0.001717,0.004588,-0.006738,-0.000151,0.001677,0.003174,0.016724
可转债,2.0,0.023078,0.032637,0.000000,0.011539,0.023078,0.034617,0.046156
固收+(0+),27.0,0.003820,0.006526,-0.008270,0.000380,0.003932,0.008539,0.016914
固收+(15-25),27.0,0.007280,0.012826,-0.021103,0.000788,0.007856,0.015389,0.044104
固收+(25-35),27.0,0.004349,0.022177,-0.036905,-0.004778,-0.000102,0.011188,0.067530
固收+(35+),27.0,0.004379,0.025889,-0.062367,-0.011994,0.006006,0.016819,0.053051
固收+(5-15),27.0,0.005636,0.011498,-0.027375,-0.000723,0.004652,0.015405,0.025597
短债,27.0,0.000867,0.004399,-0.007085,-0.000640,0.000266,0.002057,0.014635
细分,27.0,0.007905,0.015161,-0.016174,-0.000458,0.004648,0.011900,0.050419
